In [4]:
import telebot, wikipedia, re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
import datetime


bot = telebot.TeleBot('2130329456:AAGRRUymCdLRgfm40dOBgOIyNJdwzDjRN10')

wikipedia.set_lang("ru")

def greeting():
    """
    Функция приветствия в зависимости от времени суток
    """
    opts = { "hey" : ('Привет! ', 'Здравствуйте! ', 'Доброе утро! ', 'Добрый день! ', 'Добрый вечер! ', 'Доброй ночи! ')}

    now = datetime.datetime.now()
    #now += timedelta(hours = 3) # можешь проверить тут
    #print(now)
    if now.hour > 4 and now.hour <= 12 :
        greet = opts["hey"][2]
    if now.hour > 12 and now.hour <= 16 :
        greet = opts["hey"][3]
    if now.hour > 16 and now.hour <= 24 :
        greet = opts["hey"][4]
    if now.hour >= 0 and now.hour <= 4 :
        greet = opts["hey"][5]
    return greet



alphabet = ' 1234567890-йцукенгшщзхъфывапролджэячсмитьбюёqwertyuiopasdfghjklzxcvbnm?%.,()!:;'

def clean_str(r):
    r = r.lower()
    r = [c for c in r if c in alphabet]
    return ''.join(r)

def update():
    with open('chat_bot_BD.txt', encoding='utf-8') as f:
        content = f.read()
    
    blocks = content.split('\n')
    dataset = []
    
    for block in blocks:
        replicas = block.split('\\')[:2]
        if len(replicas) == 2:
            pair = [clean_str(replicas[0]), clean_str(replicas[1])]
            if pair[0] and pair[1]:
                dataset.append(pair)
#      как выглядит база
#     [question, answer]
#     [['привет', 'здравствуйте!'],
#      ['как дела', 'хорошо.']]
    
    
    # создаём два списка
    X_text = []
    y = []
    
    for question, answer in dataset[:10000]:
        X_text.append(question) # список вопросов
        y += [answer] # список ответов
    
    
    # машинное обучение 
    global vectorizer
    vectorizer = CountVectorizer() # CountVectorizer по умолчанию  производит токенизацию и выкидывает слова с длиной меньшей чем два.
    X = vectorizer.fit_transform(X_text) #
    
    #scores_list = []
    global clf
    #clfs = [LogisticRegression(),  GradientBoostingClassifier(),  RandomForestClassifier(), 
    # BaggingClassifier(), ExtraTreesClassifier(),  AdaBoostClassifier()]
    #for clf in clfs:
    #clf = LogisticRegression()
    clf = GradientBoostingClassifier()    
    clf.fit(X, y)
# #         n=2
# #         while (X.shape[0]//n > n) and (n<=5):
# #             n+=1
# #         try: 
# #             scores = cross_val_score(clf, X, y, cv=n-2)
# #             scores_list.append(scores.mean())
# #         except:
#         scores = cross_val_score(clf, X, y, cv=3)
#         scores_list.append(scores.mean())
#         ind_best = scores_list.index(max(scores_list))
#     clf = clfs[ind_best]
#     print(clf)    

update()

def get_generative_replica(text): #генерим ответ на запрос
    text_vector = vectorizer.transform([text]).toarray()[0] # векторизируем запрос
    question = clf.predict([text_vector])[0] #по вектору вычисляем ближайший вопрос
    return question

def getwiki(s):
    try:
        ny = wikipedia.page(s)
        wikitext=ny.content[:1000]
        wikimas=wikitext.split('.')
        wikimas = wikimas[:-1]
        wikitext2 = ''
        for x in wikimas:
            if not('==' in x):
                if(len((x.strip()))>3):
                    wikitext2=wikitext2+x+'.'
            else:
                break
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\{[^\{\}]*\}', '', wikitext2)
        return wikitext2
    except Exception as e:
        return 'В энциклопедии нет информации об этом'

@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id,"Здравствуйте, Сэр.")

question = ""

@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    command = message.text.lower()
    if command in ('хай','хей','хаюшки','здрасьте', 'салам','привет','приветик', 'здорово','здрасьте','здравствуйте', \
                'доброе утро', 'добрый день', 'добрый вечер', 'доброй ночи','приветствую','здрасте',\
               'салют', 'мое почтение', 'наше вам', 'здоровеньки','здравия желаю', 'чао', 'хелло', 'физкульт', 'шалом'):
        bot.send_message(message.from_user.id, greeting()) 
    elif command =="--":
        bot.send_message(message.from_user.id, "а как правильно?")
        bot.register_next_step_handler(message, wrong)
    else:
        global question
        question = command
        reply = get_generative_replica(command)
        print(reply)        
        if "вики" in reply:
            bot.send_message(message.from_user.id, getwiki(command))
        else:
            bot.send_message(message.from_user.id, reply)

def wrong(message): # функция записывает правильный ответ в файл
    a = "{}\{}\n".format(question, message.text.lower())
        #{question}\{message.text.lower()}.\n"
    with open('chat_bot_BD.txt', "a", encoding='utf-8') as f:
        f.write(a)
    bot.send_message(message.from_user.id, "Готово")
    update()

bot.polling(none_stop=True)


KeyboardInterrupt: 

In [6]:
# Получим список возможных параметров выбранной модели clf = GradientBoostingClassifier()    
clf.get_params()#.keys() 

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'log_loss',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [8]:
with open('chat_bot_BD.txt', encoding='utf-8') as f:
    content = f.read()

blocks = content.split('\n')
dataset = []

In [9]:
blocks

['привет\\здравствуйте!',
 'как дела\\хорошо...',
 'кто ты\\я чат-бот',
 'ты знаешь колю\\да ',
 'кошка\\маленькое пушистое животное, которое мяукает',
 'кот\\кошка мужского пола',
 'жмот\\жадный человек',
 'что нового\\новостей нет',
 'что ты можешь\\я могу отвечать на ваши сообщения',
 'круто\\а вы думали!',
 'поговорим\\конечно',
 'драндулет\\ расдолбанный, старый велосипед',
 'кратер\\Отверстие в вершине вулкана, через которое извергается лава.',
 'геродот\\вики',
 'интеграл\\одно из важнейших понятий математического анализа, которое возникает при решении задач: о нахождении площади под кривой; пройденного пути при неравномерном движении; массы неоднородного тела, и тому подобных; а также в задаче о восстановлении функции по её производной',
 'пижон\\человек у которого количество жен равно числу пи = 3.14',
 'пеньюар\\дурак из южной африки',
 'Автомат\\Самокритика',
 'Алимент\\Гастарбайтер, работающий в полиции',
 'Армани\\ Военный бюджет; Армянские деньги; Рука Ани',
 'Бабочки\\Же

In [14]:
blocks

['привет\\здравствуйте!',
 'как дела\\хорошо...',
 'кто ты\\я чат-бот',
 'ты знаешь колю\\да ',
 'кошка\\маленькое пушистое животное, которое мяукает',
 'кот\\кошка мужского пола',
 'жмот\\жадный человек',
 'что нового\\новостей нет',
 'что ты можешь\\я могу отвечать на ваши сообщения',
 'круто\\а вы думали!',
 'поговорим\\конечно',
 'драндулет\\ расдолбанный, старый велосипед',
 'кратер\\Отверстие в вершине вулкана, через которое извергается лава.',
 'геродот\\вики',
 'интеграл\\одно из важнейших понятий математического анализа, которое возникает при решении задач: о нахождении площади под кривой; пройденного пути при неравномерном движении; массы неоднородного тела, и тому подобных; а также в задаче о восстановлении функции по её производной',
 'пижон\\человек у которого количество жен равно числу пи = 3.14',
 'пеньюар\\дурак из южной африки',
 '',
 '']

In [61]:
for block in blocks:
    replicas = block.split('\\')[:2]
    print(replicas)
    if len(replicas) == 2:
        pair = [clean_str(replicas[0]), clean_str(replicas[1])]
        if pair[0] and pair[1]:
            dataset.append(pair)

['привет', 'здравствуйте!']
['как дела', 'хорошо.']
['кто ты', 'я чат-бот.']
['ты знаешь колю', 'да. ']
['кошка', 'маленькое пушистое животное, которое мяукает']
['кот', 'кошка мужского пола']
['жмот', 'жадный человек']
['что нового', 'новостей нет']
['что ты можешь', 'я могу отвечать на ваши сообщения']
['круто', 'а вы думали!']
['поговорим', 'конечно']
['вики', 'вики']
['геродот', 'вики']
['драндулет', 'вики']
['драндулет', ' расдолбанный, старый велосипед']
['кратер', 'Отверстие в вершине вулкана, через к-рое извергается лава.']
['']


In [62]:
dataset

[['привет', 'здравствуйте!'],
 ['как дела', 'хорошо.'],
 ['кто ты', 'я чат-бот.'],
 ['ты знаешь колю', 'да. '],
 ['кошка', 'маленькое пушистое животное, которое мяукает'],
 ['кот', 'кошка мужского пола'],
 ['жмот', 'жадный человек'],
 ['что нового', 'новостей нет'],
 ['что ты можешь', 'я могу отвечать на ваши сообщения'],
 ['круто', 'а вы думали!'],
 ['поговорим', 'конечно'],
 ['вики', 'вики'],
 ['геродот', 'вики'],
 ['драндулет', 'вики'],
 ['драндулет', ' расдолбанный, старый велосипед'],
 ['кратер', 'отверстие в вершине вулкана, через к-рое извергается лава.']]

In [63]:
X_text = []
y = []
for question, answer in dataset[:10000]:
    print(question, '-',answer)
    X_text.append(question)
    y += [answer]

привет - здравствуйте!
как дела - хорошо.
кто ты - я чат-бот.
ты знаешь колю - да. 
кошка - маленькое пушистое животное, которое мяукает
кот - кошка мужского пола
жмот - жадный человек
что нового - новостей нет
что ты можешь - я могу отвечать на ваши сообщения
круто - а вы думали!
поговорим - конечно
вики - вики
геродот - вики
драндулет - вики
драндулет -  расдолбанный, старый велосипед
кратер - отверстие в вершине вулкана, через к-рое извергается лава.


In [64]:
X_text

['привет',
 'как дела',
 'кто ты',
 'ты знаешь колю',
 'кошка',
 'кот',
 'жмот',
 'что нового',
 'что ты можешь',
 'круто',
 'поговорим',
 'вики',
 'геродот',
 'драндулет',
 'драндулет',
 'кратер']

In [65]:
y

['здравствуйте!',
 'хорошо.',
 'я чат-бот.',
 'да. ',
 'маленькое пушистое животное, которое мяукает',
 'кошка мужского пола',
 'жадный человек',
 'новостей нет',
 'я могу отвечать на ваши сообщения',
 'а вы думали!',
 'конечно',
 'вики',
 'вики',
 'вики',
 ' расдолбанный, старый велосипед',
 'отверстие в вершине вулкана, через к-рое извергается лава.']

In [66]:
vectorizer = CountVectorizer() # CountVectorizer по умолчанию  производит токенизацию и выкидывает слова с длиной меньшей чем два.
X = vectorizer.fit_transform(X_text)

In [38]:
X

<7x10 sparse matrix of type '<class 'numpy.int64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [67]:
clf = LogisticRegression()
clf.fit(X, y)

LogisticRegression()

In [73]:
text = "кратер"
text_vector = vectorizer.transform([text]).toarray()[0]
question = clf.predict([text_vector])[0]
question

'вики'

In [45]:
vectorizer.transform([text])

<1x10 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [42]:
vectorizer.transform([text]).toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]], dtype=int64)

In [44]:
clf.predict([text_vector])

array(['здравствуйте!'], dtype='<U44')

### описание
https://habr.com/ru/post/667008/
Давно хотел сделать своего собственного Jarvis. Недавно удалась свободная минутка и я его сделал. Он умеет переписываться с Вами, а также искать ответы на Ваши вопросы в Wikipedia. Для его реализации я использовал язык Python.

Для начала установим все необходимые библиотеки. Их три: pyTelegramBotAPI, scikit-learn, а также Wikipedia. Устанавливаются они просто:

pip install pyTelegramBotAPI

pip install Wikipedia

pip install scikit-learn

После установки всех библиотек приступаем к разработке. Для начала импортируем все библиотеки, установим язык для Википедии и подключим телеграмм бота

import telebot, wikipedia, re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

wikipedia.set_lang("ru")
bot = telebot.TeleBot('Ваш ключ, полученный от BotFather')

Теперь напишем код, для очистки всех ненужных нам знаков, которые вводит пользователь:

def clean_str(r):
	r = r.lower()
	r = [c for c in r if c in alphabet]
	return ''.join(r)

alphabet = ' 1234567890-йцукенгшщзхъфывапролджэячсмитьбюёqwertyuiopasdfghjklzxcvbnm?%.,()!:;'

Также Вам необходимо создать в папке, где находится Ваш код файл dialogues.txt, в нем мы будем создавать реплики на которые должен отвечать бот. Вот пример данного файла:

привет\здравствуйте!
как дела\хорошо.
кто ты\я Джарвис.

Строка до знака \ означает вопрос пользователя, а после ответ нашего бота. После чего напишем такой код в наш файл с ботом:

def update():
	with open('dialogues.txt', encoding='utf-8') as f:
		content = f.read()
	
	blocks = content.split('\n')
	dataset = []
	
	for block in blocks:
		replicas = block.split('\\')[:2]
		if len(replicas) == 2:
			pair = [clean_str(replicas[0]), clean_str(replicas[1])]
			if pair[0] and pair[1]:
				dataset.append(pair)
	
	X_text = []
	y = []
	
	for question, answer in dataset[:10000]:
		X_text.append(question)
		y += [answer]
	
	global vectorizer
	vectorizer = CountVectorizer()
	X = vectorizer.fit_transform(X_text)
	
	global clf
	clf = LogisticRegression()
	clf.fit(X, y)

update()

Этот кусок кода читает файл dialogues.txt, потом превращает реплики в так называемые вектора, с помощью которых наш бот будет искать наиболее подходящий ответ к заданному нами вопросу. Например, если Вы написали в файле dialogues.txt вопрос "Ты знаешь Аню", а ответ на него "Да, конечно", то бот будет отвечать также и на похожие вопросы, например "Ты знаешь Васю".

Теперь напишем кусок кода, который будет генерировать ответы на основе векторов:

def get_generative_replica(text):
	text_vector = vectorizer.transform([text]).toarray()[0]
	question = clf.predict([text_vector])[0]
	return question

Этот кусок кода принимает вопрос от пользователя и возвращает ответ от бота.

Теперь напишем функцию для поиска информации в Википедии:

def getwiki(s):
    try:
        ny = wikipedia.page(s)
        wikitext=ny.content[:1000]
        wikimas=wikitext.split('.')
        wikimas = wikimas[:-1]
        wikitext2 = ''
        for x in wikimas:
            if not('==' in x):
                if(len((x.strip()))>3):
                   wikitext2=wikitext2+x+'.'
            else:
                break
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\{[^\{\}]*\}', '', wikitext2)
        return wikitext2
    except Exception as e:
        return 'В Википедии нет информации об этом'

Этот кусок кода получает вопрос пользователя, потом ищет ответ на него в Википедии и если ответ найден, то отдает его пользователю, а если ответ не найден, то пишет, что "В Википедии нет информации об этом".

Теперь пишем последний кусок кода:

@bot.message_handler(commands=['start'])
def start_message(message):
	bot.send_message(message.chat.id,"Здравствуйте, Сэр.")

question = ""

@bot.message_handler(content_types=['text'])
def get_text_messages(message):
	command = message.text.lower()
	if command =="не так":
		bot.send_message(message.from_user.id, "а как?")
		bot.register_next_step_handler(message, wrong)
	else:
		global question
		question = command
		reply = get_generative_replica(command)
		if reply=="вики ":
			bot.send_message(message.from_user.id, getwiki(command))
		else:
			bot.send_message(message.from_user.id, reply)

def wrong(message):
	a = f"{question}\{message.text.lower()} \n"
	with open('dialogues.txt', "a", encoding='utf-8') as f:
		f.write(a)
	bot.send_message(message.from_user.id, "Готово")
	update()

В этом куске кода телеграмм бот при получении сообщения от пользователя отвечает на него и если ответ не верный, то пользователь пишет "не так". Если бот получает сообщение "не так", то он берет последний вопрос пользователя и спрашивает "а как?", после чего пользователь должен отправить ему правильный ответ. После этого бот обновляет свою базу данных вопросов и ответов и при следующих вопросах пользователя отвечает на них правильно. И если ответ на вопрос бот должен был взять из Википедии, то пользователь в ответ на вопрос "а как?", должен написать "wiki". Осталось в конце приписать строчку:

bot.polling(none_stop=True)

И можно запускать и тестировать бота.